In [12]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

from models.StackedAutoEncoder import SimpleAutoEncoder
from models.LSTM import LSTM, GRU

## Data Prep

In [22]:
X_train = torch.tensor(np.load(Path("model_data/x_train.npy")), dtype=torch.float32)
X_val = torch.tensor(np.load(Path("model_data/x_val.npy")), dtype=torch.float32)
X_test = torch.tensor(np.load(Path("model_data/x_test.npy")), dtype=torch.float32)
Y_train = torch.tensor(np.load(Path("model_data/y_train.npy")), dtype=torch.float32)
Y_val = torch.tensor(np.load(Path("model_data/y_val.npy")), dtype=torch.float32)
Y_test = torch.tensor(np.load(Path("model_data/y_test.npy")), dtype=torch.float32)
X_train.shape

torch.Size([482, 20, 20])

In [23]:
input_size = 20
hidden_size = 10
num_layers = 4
output_size = 1
batch_size = 60
num_epochs = 5_000
lr = 0.05

## SAE

In [ ]:
model = SimpleAutoEncoder(input_size=input_size, hidden_size=hidden_size)
model.fit(X_train, epochs=5_000, lr=0.05, VERBOSE=True)

x_train_sae = model.forward(X_train)
x_val_sae = model.forward(X_val)
x_test_sae = model.forward(X_test)
x_train_sae.shape

In [ ]:
np.save(Path("model_data/x_train_sae.npy"), x_train_sae.detach().numpy())
np.save(Path("model_data/x_val_sae.npy"), x_val_sae.detach().numpy())
np.save(Path("model_data/x_test_sae.npy"), x_test_sae.detach().numpy())

In [24]:
X_train = torch.tensor(np.load(Path("model_data/x_train_sae.npy")), dtype=torch.float32)
X_val = torch.tensor(np.load(Path("model_data/x_val_sae.npy")), dtype=torch.float32)
X_test = torch.tensor(np.load(Path("model_data/x_test_sae.npy")), dtype=torch.float32)
X_train.shape

torch.Size([482, 20, 20])

## LSTM (SAE-LSTM)

In [25]:
model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, output_size=output_size)
# model.fit(x_train, y_train)

criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr)
train_dataset = TensorDataset(X_train, Y_train)
train_dataset = train_dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, Y_val)
val_dataset = val_dataset
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_hist = []
val_hist = []

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.float()
        predictions = model(batch_X.float())

        predictions = predictions.float()  
        batch_y = batch_y.float()  

        loss = criterion(predictions, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    optimizer.step()
    average_loss = total_loss/len(train_loader)
    train_hist.append(average_loss)

    model.eval()
    with torch.no_grad():
        total_val_loss = 0.0

        for batch_X_val, batch_y_val in val_loader:
            predictions_val = model(batch_X_val.float())

            lossmsev = criterion(predictions_val, batch_y_val.float())
            val_loss = lossmsev
            total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(val_loader)
        val_hist.append(average_val_loss)

    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.3f}, MSE Loss: {loss:.6f}, Val Loss: {average_val_loss:.6f}, Learning Rate: {optimizer.param_groups[0]["lr"]}')

/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([60, 20])) that is different to the input size (torch.Size([60, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2, 20])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([41, 20])) that is different to the input size (torch.Size([41, 1])). This will likely lead to incorrect results due to 

Epoch [1/5000] - Training Loss: 0.100, MSE Loss: 0.110702, Val Loss: 0.049527, Learning Rate: 0.05
Epoch [2/5000] - Training Loss: 0.054, MSE Loss: 0.030729, Val Loss: 0.084166, Learning Rate: 0.05
Epoch [3/5000] - Training Loss: 0.047, MSE Loss: 0.116117, Val Loss: 0.064315, Learning Rate: 0.05
Epoch [4/5000] - Training Loss: 0.062, MSE Loss: 0.058550, Val Loss: 0.074247, Learning Rate: 0.05
Epoch [5/5000] - Training Loss: 0.045, MSE Loss: 0.030533, Val Loss: 0.072194, Learning Rate: 0.05
Epoch [6/5000] - Training Loss: 0.041, MSE Loss: 0.039220, Val Loss: 0.066400, Learning Rate: 0.05
Epoch [7/5000] - Training Loss: 0.035, MSE Loss: 0.018305, Val Loss: 0.091058, Learning Rate: 0.05
Epoch [8/5000] - Training Loss: 0.031, MSE Loss: 0.019059, Val Loss: 0.075540, Learning Rate: 0.05
Epoch [9/5000] - Training Loss: 0.033, MSE Loss: 0.020765, Val Loss: 0.104279, Learning Rate: 0.05
Epoch [10/5000] - Training Loss: 0.027, MSE Loss: 0.009161, Val Loss: 0.104474, Learning Rate: 0.05
Epoch [11

### LSTM Results

In [26]:
def results(model, X_test, Y_test):
    """"""
    model.eval()
    pred = model(X_test).detach().numpy()
    print("MSE:", mean_squared_error(Y_test[:, -1], pred))
    print("r2:", r2_score(Y_test[:, -1], pred))
    print("R1:", np.corrcoef(pred.T, Y_test[:, -1:].detach().numpy().T))

lstm = model

In [27]:
results(lstm, X_train, Y_train)
results(lstm, X_val, Y_val)
results(lstm, X_test, Y_test)

MSE: 0.020788334
r2: 0.6629560224868144
R1: [[1.         0.82995545]
 [0.82995545 1.        ]]
MSE: 0.12922236
r2: -1.0280135993907034
R1: [[ 1.         -0.59802626]
 [-0.59802626  1.        ]]
MSE: 0.18262948
r2: -0.7587598981772627
R1: [[ 1.         -0.67824831]
 [-0.67824831  1.        ]]


# GRU (SAE-GRU)

In [28]:
model = GRU(input_size=input_size, hidden_size=hidden_size, num_layers=1, output_size=output_size)
# model.fit(X_train, Y_train, epochs=1_000)

criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr)
train_dataset = TensorDataset(X_train, Y_train)
train_dataset = train_dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, Y_val)
val_dataset = val_dataset
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_hist = []
val_hist = []

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.float()
        predictions = model(batch_X.float())

        predictions = predictions.float()  
        batch_y = batch_y.float()  

        loss = criterion(predictions, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    optimizer.step()
    average_loss = total_loss/len(train_loader)
    train_hist.append(average_loss)

    model.eval()
    with torch.no_grad():
        total_val_loss = 0.0

        for batch_X_val, batch_y_val in val_loader:
            predictions_val = model(batch_X_val.float())

            lossmsev = criterion(predictions_val[:, -1:], batch_y_val.float())
            val_loss = lossmsev
            total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(val_loader)
        val_hist.append(average_val_loss)

    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.3f}, MSE Loss: {loss:.6f}, Val Loss: {average_val_loss:.6f}, Learning Rate: {optimizer.param_groups[0]["lr"]}')

/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([60, 20])) that is different to the input size (torch.Size([60, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2, 20])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([41, 20])) that is different to the input size (torch.Size([41, 1])). This will likely lead to incorrect results due to 

Epoch [1/5000] - Training Loss: 0.114, MSE Loss: 0.053424, Val Loss: 0.091154, Learning Rate: 0.05
Epoch [2/5000] - Training Loss: 0.064, MSE Loss: 0.052727, Val Loss: 0.114800, Learning Rate: 0.05
Epoch [3/5000] - Training Loss: 0.080, MSE Loss: 0.009127, Val Loss: 0.126195, Learning Rate: 0.05
Epoch [4/5000] - Training Loss: 0.038, MSE Loss: 0.027651, Val Loss: 0.192331, Learning Rate: 0.05
Epoch [5/5000] - Training Loss: 0.054, MSE Loss: 0.005300, Val Loss: 0.078196, Learning Rate: 0.05
Epoch [6/5000] - Training Loss: 0.030, MSE Loss: 0.011301, Val Loss: 0.096153, Learning Rate: 0.05
Epoch [7/5000] - Training Loss: 0.030, MSE Loss: 0.031741, Val Loss: 0.091646, Learning Rate: 0.05
Epoch [8/5000] - Training Loss: 0.028, MSE Loss: 0.041317, Val Loss: 0.145021, Learning Rate: 0.05
Epoch [9/5000] - Training Loss: 0.038, MSE Loss: 0.023810, Val Loss: 0.142261, Learning Rate: 0.05
Epoch [10/5000] - Training Loss: 0.033, MSE Loss: 0.003784, Val Loss: 0.138627, Learning Rate: 0.05
Epoch [11

### GRU Results

In [29]:
gru = model

In [30]:
results(gru, X_train, Y_train)
results(gru, X_val, Y_val)
results(gru, X_test, Y_test)

MSE: 0.03336084
r2: 0.4591164613291221
R1: [[1.         0.71446008]
 [0.71446008 1.        ]]
MSE: 0.17497209
r2: -1.7460088642203333
R1: [[ 1.         -0.83573377]
 [-0.83573377  1.        ]]
MSE: 0.20389211
r2: -0.9635235311509933
R1: [[ 1.         -0.54123572]
 [-0.54123572  1.        ]]
